1. Pick a Potsdam scene
2. Pick a class
3. Define raster source
4. Define label source such that there are only 2 classes
5. Define a pre-labeled AOI
5. Define a DataConfig
6. Define a LearnerConfig
7. Instantiate a Learner
8. Train on scene
9. Predict on scene
10. Use probability raster to compute task priority scores
11. Use scores to select tasks to "label"
12. Add the newly "labeled" tasks to the AOI
13. Got to 5

In [1]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 579 kB/s            
     |████████████████████████████████| 16.7 MB 7.1 MB/s            
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [ ]:
from typing import (List, Tuple)
import geopandas as gpd

from shapely.geometry import Polygon

from rastervision.core.box import Box
from rastervision.core.data import (ClassConfig, CRSTransformer,
                                    GeoJSONVectorSourceConfig, RasterioSource,
                                    RasterizedSource, Scene,
                                    SemanticSegmentationLabelSource)
from rastervision.pytorch_learner import (
    SemanticSegmentationGeoDataConfig, SemanticSegmentationLearner, 
    SemanticSegmentationSlidingWindowGeoDataset)


In [ ]:
def label_tasks(task_grid_with_scores: gpd.GeoDataFrame, 
                n: int) -> Tuple[str, gpd.GeoDataFrame]:
    pass

In [ ]:
def get_labeled_tasks(task_grid: gpd.GeoDataFrame) -> List[Polygon]:
    pass

In [ ]:
def get_img_info(uri: str) -> dict:
    with open(uri, 'r') as f:
        img_info = json.load(f)['results'][0]
    return img_info

def get_task_grid(uri: str) -> gpd.GeoDataFrame:
    pass

In [ ]:
def make_scene(scene_id: str, 
               class_config: ClassConfig,
               img_info: dict, 
               labels_uri: str, 
               aoi: List[Polygon]) -> Scene:
    raster_source = make_raster_source(img_info)
    
    label_source = make_label_source(
        labels_uri=labels_uri,
        class_config=class_config,
        extent=raster_source.get_extent(),
        crs_transformer=raster_source.get_crs_transformer())
    
    scene = Scene(raster_source=raster_source, label_source=label_source)
    return scene

def make_raster_source(img_info: dict) -> RasterioSource:
    img_uri = img_info['ingestLocation']
    raster_source = RasterioSource(uris=[img_uri])
    return raster_source

def make_label_source(labels_uri: str,
                      class_config: ClassConfig,
                      extent: Box,
                      crs_transformer: CRSTransformer
                     ) -> SemanticSegmentationLabelSource:
    geojson_cfg = GeoJSONVectorSourceConfig(uri=labels_uri,
                                            default_class_id=1,
                                            ignore_crs_field=True)
    vector_source = geojson_cfg.build(class_config, crs_transformer)
    
    label_source = SemanticSegmentationLabelSource(
        raster_source=RasterizedSource(
            vector_source=vector_source, 
            background_class_id=0,
            extent=extent,
            crs_transformer=crs_transformer),
        null_class_id=0)
    return label_source

In [ ]:
def train(scene: Scene) -> SemanticSegmentationLearner:
    pass

In [ ]:
def predict(learner: SemanticSegmentationLearner, 
            scene: Scene, 
            output_dir: str) -> None:
    pass

In [ ]:
def compute_priority_scores(task_grid: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    pass

In [ ]:
def al_step(iter_num: int,
            class_config: ClassConfig,
            img_info_uri: str, 
            labels_uri: str, 
            task_grid: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    labeled_regions = get_labeled_tasks(task_grid)
    scene = make_scene(
        scene_id=f'scene-iter-{iter_num}',
        class_config=class_config,
        img_info_uri=img_info_uri
        labels_uri=labels_uri,
        aoi=labeled_regions)
    learner = train(scene)
    pred_dir = f'preds-iter-{iter_num}'
    predict(learner, scene, output_dir=pred_dir)
    task_grid_with_scores = compute_priority_scores(task_grid)
    return task_grid_with_scores

In [ ]:
def al_loop(niters: int, 
            class_config: ClassConfig,
            img_info: dict, 
            task_grid: gpd.GeoDataFrame,
            ntasks_per_iter: int) -> None:
    for i in range(niters):
        labels_uri, task_grid = label_tasks(task_grid, n=ntasks_per_iter)
        task_grid = al_step(iter_num=i,
                            img_info=img_info,
                            labels_uri=labels_uri,
                            task_grid=task_grid)

In [ ]:
img_info = get_img_info('')
task_grid = get_task_grid('')

al_loop(
    niters=1,
    class_config=class_config,
    img_info=img_info, 
    task_grid=task_grid,
    ntasks_per_iter=1)